In [2]:
import cv2
import os
import numpy as np
import random
import shutil
from tqdm import tqdm

class DatasetAugmentor:
    def __init__(self, input_dir, output_dir, target_images=560):
        self.input_dir = input_dir
        self.output_dir = output_dir
        self.target_images = target_images
        
        # Augmentation parameters
        self.rotation_angles = [-15, -10, -5, 5, 10, 15]
        self.brightness_factors = [0.7, 0.85, 1.15, 1.3]
        self.zoom_factors = [0.9, 0.95, 1.05, 1.1]
        self.shift_pixels = [-10, -5, 5, 10]
        self.noise_std = 10
        
        # Create output directory
        os.makedirs(self.output_dir, exist_ok=True)
    
    def augment_image(self, img, method):
        """Apply one augmentation method to the image."""
        h, w = img.shape[:2]
        
        try:
            if method == "flip":
                return cv2.flip(img, 1)
            elif method == "rotate":
                angle = random.choice(self.rotation_angles)
                M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1.0)
                return cv2.warpAffine(img, M, (w, h), borderMode=cv2.BORDER_REFLECT)
            elif method == "brightness":
                factor = random.choice(self.brightness_factors)
                hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
                hsv[:, :, 2] = cv2.multiply(hsv[:, :, 2], factor)
                return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
            elif method == "zoom":
                factor = random.choice(self.zoom_factors)
                new_w, new_h = int(w * factor), int(h * factor)
                resized = cv2.resize(img, (new_w, new_h))
                if factor > 1.0:
                    start_x = (new_w - w) // 2
                    start_y = (new_h - h) // 2
                    return resized[start_y:start_y+h, start_x:start_x+w]
                else:
                    padded = np.zeros_like(img)
                    start_x = (w - new_w) // 2
                    start_y = (h - new_h) // 2
                    padded[start_y:start_y+new_h, start_x:start_x+new_w] = resized
                    return padded
            elif method == "shift":
                tx = random.choice(self.shift_pixels)
                ty = random.choice(self.shift_pixels)
                M = np.float32([[1, 0, tx], [0, 1, ty]])
                return cv2.warpAffine(img, M, (w, h), borderMode=cv2.BORDER_REFLECT)
            elif method == "noise":
                noise = np.random.normal(0, self.noise_std, img.shape).astype(np.uint8)
                return cv2.add(img, noise)
            elif method == "blur":
                return cv2.GaussianBlur(img, (3, 3), 0)
            else:
                return img
        except Exception as e:
            print(f"Error in augmentation {method}: {e}")
            return img
    
    def get_available_methods(self):
        """Return available augmentation methods."""
        return ["flip", "rotate", "brightness", "zoom", "shift", "noise", "blur"]
    
    def augment_dataset(self):
        """Main method to augment the entire dataset."""
        person_folders = [f for f in os.listdir(self.input_dir) 
                         if os.path.isdir(os.path.join(self.input_dir, f))]
        
        print(f"Found {len(person_folders)} person folders")
        print(f"Target images per class: {self.target_images}")
        
        stats = []
        
        for person_name in tqdm(person_folders, desc="Processing classes"):
            person_path = os.path.join(self.input_dir, person_name)
            output_person_path = os.path.join(self.output_dir, person_name)
            os.makedirs(output_person_path, exist_ok=True)
            
            # Get original images
            images = [f for f in os.listdir(person_path) 
                     if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
            num_original = len(images)
            
            # Copy original images
            for img_name in images:
                src_path = os.path.join(person_path, img_name)
                dst_path = os.path.join(output_person_path, img_name)
                shutil.copy2(src_path, dst_path)
            
            if num_original >= self.target_images:
                stats.append((person_name, num_original, num_original, 0))
                continue
            
            needed = self.target_images - num_original
            augmentation_count = 0
            methods = self.get_available_methods()
            
            # Generate augmented images
            while augmentation_count < needed:
                for img_name in images:
                    if augmentation_count >= needed:
                        break
                    
                    img_path = os.path.join(person_path, img_name)
                    img = cv2.imread(img_path)
                    if img is None:
                        continue
                    
                    # Apply augmentation
                    method = random.choice(methods)
                    aug_img = self.augment_image(img, method)
                    
                    # Save augmented image
                    base_name = os.path.splitext(img_name)[0]
                    out_name = f"{base_name}_aug{augmentation_count:04d}.jpg"
                    out_path = os.path.join(output_person_path, out_name)
                    
                    cv2.imwrite(out_path, aug_img)
                    augmentation_count += 1
            
            final_count = num_original + augmentation_count
            stats.append((person_name, num_original, final_count, augmentation_count))
        
        # Print summary
        self._print_summary(stats)
    
    def _print_summary(self, stats):
        """Print augmentation summary."""
        print("\n" + "="*50)
        print("AUGMENTATION SUMMARY")
        print("="*50)
        total_original = sum(s[1] for s in stats)
        total_final = sum(s[2] for s in stats)
        total_augmented = sum(s[3] for s in stats)
        
        print(f"Total original images: {total_original}")
        print(f"Total augmented images: {total_augmented}")
        print(f"Total final images: {total_final}")
        print(f"Augmentation ratio: {total_augmented/total_original:.2f}x")
        print(f"Output directory: {self.output_dir}")

# Usage
if __name__ == "__main__":
    # Configuration
    input_dir = r"D:\Final_Semester_Project\AI_Attendance_System\AI_And_ML_Model\DataSets\Recognize"
    output_dir = r"D:\Final_Semester_Project\AI_Attendance_System\AI_And_ML_Model\DataSets\RecognizeAgumented"
    target_images = 700
    
    # Initialize and run augmentor
    augmentor = DatasetAugmentor(input_dir, output_dir, target_images)
    augmentor.augment_dataset()

Found 58 person folders
Target images per class: 700


Processing classes: 100%|██████████████████████████████████████████████████████████████| 58/58 [02:08<00:00,  2.22s/it]


AUGMENTATION SUMMARY
Total original images: 4022
Total augmented images: 36578
Total final images: 40600
Augmentation ratio: 9.09x
Output directory: D:\Final_Semester_Project\AI_Attendance_System\AI_And_ML_Model\DataSets\RecognizeAgumented


In [3]:
pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
